In [5]:
from gptcher.gpt_client import supabase
from plotly import express as px
import pandas as pd

def get_all(table):
    rows = []
    cursor = None
    start = 0
    limit = 1000
    while True:
        new_rows = supabase.table(table).select("*").range(start, limit).execute().data
        rows.extend(new_rows)
        if len(new_rows) < 1000:
            break
        start += 1000
        limit += 1000
    df = pd.DataFrame(rows)
    # Transform the date to a datetime object
    if 'created_at' in df.columns:
        df["created_at"] = pd.to_datetime(df["created_at"])
    return df



messages = get_all("messages")
# remove columns: user_id
messages = messages.drop(columns=["user_id"])
users = get_all("users")
sessions = get_all("session")
# merge sessions and messages
messages = messages.merge(sessions, left_on="session", right_on="id", suffixes=("", "_session"))


In [6]:

# Plot the number of new users per day (bar plot)
users["created_at"] = pd.to_datetime(users["created_at"])
users["created_at"] = users["created_at"].dt.date
users_per_day = users.groupby("created_at").count()
fig = px.bar(users_per_day, x=users_per_day.index, y="user_id")
fig.show()

In [4]:
messages.columns

Index(['id_message', 'created_at_message', 'text', 'text_en',
       'text_translated', 'voice_url', 'session', 'sender', 'evaluation',
       'id_session', 'created_at_session', 'type', 'user_id', 'context'],
      dtype='object')

In [7]:
# Plot the number of messages per day (bar plot)
messages["created_at"] = pd.to_datetime(messages["created_at"])
messages["created_at"] = messages["created_at"].dt.date
messages_per_day = messages.groupby(["created_at", "sender"]).count().reset_index()
fig = px.bar(messages_per_day, x=messages_per_day.index, y="id", color="sender")
fig.show()
messages.groupby("sender").count()

,id,created_at,text,text_en,text_translated,voice_url,session,evaluation,id_session,created_at_session,type,user_id,context
sender,,,,,,,,,,,,,
Student,7547,7547,7532,1597,7343,0,7547,5210,7547,7547,7547,7547,7547
Teacher,7105,7105,7105,5388,5388,0,7105,0,7105,7105,7105,7105,7105


In [ ]:
# How many sessions per user
sessions_per_user = sessions.groupby("user_id").count()
sessions_per_user["id"].describe()

count    343.000000
mean       2.982507
std        8.598908
min        2.000000
25%        2.000000
50%        2.000000
75%        2.000000
max      159.000000
Name: id, dtype: float64

In [11]:
# How many messages per user

messages_per_user = messages.loc[messages.sender=='Student'].groupby("user_id").count().sort_values(by="id", ascending=False)
print(messages_per_user["id"].describe())
# Plot the number of messages per user (bar plot)
fig = px.bar(messages_per_user["id"], x=messages_per_user.index, y="id")
fig.show()

count    369.000000
mean      20.452575
std       41.497231
min        1.000000
25%        4.000000
50%        9.000000
75%       22.000000
max      458.000000
Name: id, dtype: float64


In [ ]:
# Time between first and last message per user
messages_per_user = messages.loc[messages.sender=='Student'].groupby("user_id").agg({"created_at_message": ["min", "max"]})
messages_per_user["time_diff"] = (messages_per_user["created_at_message"]["max"] - messages_per_user["created_at_message"]["min"]).dt.total_seconds() / 3600
messages_per_user["time_diff"].describe()
# Plot the time between first and last message per user (bar plot)
messages_per_user = messages_per_user.sort_values(by="time_diff", ascending=False)
fig = px.bar(messages_per_user["time_diff"], x=messages_per_user.index, y="time_diff")
fig.show()
messages_per_user.head()

created_at_message  \
                                            min   
user_id                                           
1143467066tmp4 2023-01-09 19:13:25.931250+00:00   
125598527tmp4  2023-01-09 18:58:23.682908+00:00   
5834685444     2023-01-26 00:38:56.059497+00:00   
686218161      2023-01-26 00:06:52.275220+00:00   
5329634761     2023-01-26 05:12:55.186787+00:00   

                                                  time_diff  
                                            max              
user_id                                                      
1143467066tmp4 2023-01-25 14:01:11.790315+00:00  378.796072  
125598527tmp4  2023-01-21 18:01:08.795214+00:00  287.045865  
5834685444     2023-01-31 02:56:59.774861+00:00  122.301032  
686218161      2023-01-30 18:10:30.826572+00:00  114.060709  
5329634761     2023-01-30 22:32:13.194749+00:00  113.321669